In [1]:
import logging

# Cria um logger com o nome 'jupyter_logger'
logger = logging.getLogger('jupyter_logger')

# Define o nível de log para DEBUG, então todas as mensagens de log serão mostradas
logger.setLevel(logging.DEBUG)

# Cria um manipulador de log que escreve as mensagens de log na saída padrão
handler = logging.StreamHandler()

# Define o nível de log do manipulador para DEBUG
handler.setLevel(logging.DEBUG)

# Cria um formatador de log que adiciona a data e hora à mensagem de log
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Adiciona o formatador ao manipulador
handler.setFormatter(formatter)

# Adiciona o manipulador ao logger
logger.addHandler(handler)

In [2]:
from minio import Minio
from minio.error import S3Error
import os

def minion_upload(source_file,destination_file,bucket_name="raw"):
    client = Minio(endpoint="minio:9000", 
                   access_key="wFw2nWmmNHNivuqh", 
                   secret_key="Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs", 
                   secure=False)

    # Make the bucket if it doesn't exist.
    found = client.bucket_exists(bucket_name)
    if not found:
        client.make_bucket(bucket_name)
        logger.debug(f"Created bucket {bucket_name}")
    else:
        logger.debug(f"Bucket {bucket_name} already exists")

    # Upload the file, renaming it in the process
    client.fput_object(
        bucket_name, destination_file, source_file,
    )
    logger.debug(
        f"{source_file} successfully uploaded as object {destination_file} to bucket {bucket_name}"
    )
    # os.remove(source_file)

In [3]:
import json

def writing_json(data):
    data_obj = json.loads(data)
    path = f'to_process/{data_obj["uid"]}.json'

    # Abra o arquivo onde você deseja escrever
    with open(path, 'w') as f:
        # Use json.dump para escrever o objeto no arquivo
        json.dump(data_obj, f)
    return path

In [5]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
# "file:////home/user/to_process/006790cd-5c1b-47b8-9666-045f9dad6846.json"
# Crie uma sessão Spark

def executing_merge():
    spark = SparkSession.builder \
    .appName("Merge with Delta Lake") \
    .config("spark.task.maxFailures", "1") \
    .config("fs.s3a.endpoint", "http://minio:9000") \
    .config("fs.s3a.access.key", "wFw2nWmmNHNivuqh") \
    .config("fs.s3a.secret.key", "Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs") \
    .config("fs.s3a.path.style.access", "true") \
    .config("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.ssl.enabled", "false") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()
# tiAoMHTLqpoi7TKMVX
    # Carregue os novos dados
    # newData = spark.read.json(path)
    # newData = spark.createDataFrame([data])
    newData = spark.read.text("s3a://raw/user/*")

    # Verifique se a tabela Delta existe trusted
    path = "s3a://trusted/user/*"
    if DeltaTable.isDeltaTable(spark, path):
        deltaTable = DeltaTable.forPath(spark, path)
    else:
        # Se a tabela Delta não existir, crie-a
        newData.write.format("delta").save(path)
        deltaTable = DeltaTable.forPath(spark, path)

    # Realize a operação de merge
    deltaTable.alias("oldData") \
        .merge(
            newData.alias("newData"),
            "oldData.data.id = newData.id") \
        .whenMatchedUpdate(set = { 
            "first_name" : "newData.data.first_name",
            "last_name" : "newData.data.last_name",
            "email" : "newData.data.email",
            "date_of_birth" : "newData.data.date_of_birth" }) \
        .whenNotMatchedInsert(values = { 
            "id": "newData.data.id",
            "first_name" : "newData.data.first_name",
            "last_name" : "newData.datalast_name",
            "email" : "newData.data.email",
            "date_of_birth" : "newData.data.date_of_birth" }) \
        .execute()

In [6]:
executing_merge()

AnalysisException: cannot resolve oldData.id in search condition given columns oldData.value, newData.value; line 1 pos 0

In [13]:
import boto3
import os
from botocore.client import Config

# Crie uma sessão boto3 com as configurações do MinIO
s3 = boto3.resource(
    's3',
    endpoint_url='http://minio:9000',
    aws_access_key_id='wFw2nWmmNHNivuqh',
    aws_secret_access_key='Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs',
    config=Config(signature_version='s3v4'),
    region_name='us-east-1'
)
bucket = s3.Bucket('raw')
all_data=bucket.objects.all()
counter_raw = sum(1 for _ in bucket.objects.all())
logger.debug(f"reading all files {counter_raw}")
index = 0
for obj in all_data:
    key = obj.key
    body = obj.get()['Body'].read().decode('utf-8')
    logger.debug(f"Merging data {index}")
    index+=1
    executing_merge(json.loads(body))
    

folder_path = 'trusted'
bucket_trusted = s3.Bucket('trusted')
all_data_trusted=bucket.objects.all()
counter_trusted = sum(1 for _ in bucket_trusted.objects.all())


for filename in os.listdir(folder_path):
    if os.path.isfile(filename):
        absolute_path = os.path.abspath(f"{folder_path}/{filename}")
        logger.debug(absolute_path)
        minion_upload(source_file=absolute_path, destination_file=filename, bucket_name="trusted")

bucket_trusted_pos_merge = s3.Bucket('trusted')
counter_trusted_post_merge = sum(1 for _ in bucket_trusted_pos_merge.objects.all())


logger.debug(f"Before merge RAW data count {counter_raw} Trusted data count {counter_trusted}")
logger.debug(f"After merge RAW data count {counter_raw} Trusted data count {counter_trusted_post_merge}")

2024-05-20 00:08:04,682 - jupyter_logger - DEBUG - reading all files 10
DEBUG:jupyter_logger:reading all files 10
2024-05-20 00:08:04,697 - jupyter_logger - DEBUG - Merging data 0
DEBUG:jupyter_logger:Merging data 0
2024-05-20 00:08:06,672 - jupyter_logger - DEBUG - Merging data 1
DEBUG:jupyter_logger:Merging data 1
2024-05-20 00:08:07,877 - jupyter_logger - DEBUG - Merging data 2
DEBUG:jupyter_logger:Merging data 2
2024-05-20 00:08:08,911 - jupyter_logger - DEBUG - Merging data 3
DEBUG:jupyter_logger:Merging data 3
2024-05-20 00:08:09,955 - jupyter_logger - DEBUG - Merging data 4
DEBUG:jupyter_logger:Merging data 4
2024-05-20 00:08:11,042 - jupyter_logger - DEBUG - Merging data 5
DEBUG:jupyter_logger:Merging data 5
2024-05-20 00:08:12,099 - jupyter_logger - DEBUG - Merging data 6
DEBUG:jupyter_logger:Merging data 6
2024-05-20 00:08:13,103 - jupyter_logger - DEBUG - Merging data 7
DEBUG:jupyter_logger:Merging data 7
2024-05-20 00:08:14,180 - jupyter_logger - DEBUG - Merging data 8
DEBU

In [ ]:
# from pyspark.sql import SparkSession

# # Crie uma sessão Spark
# spark = SparkSession.builder \
#     .appName("Merge with Delta Lake") \
#     .config("spark.hadoop.fs.s3a.endpoint", "minio") \
#     .config("spark.hadoop.fs.s3a.access.key", "wFw2nWmmNHNivuqh") \
#     .config("spark.hadoop.fs.s3a.secret.key", "Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs") \
#     .config("spark.hadoop.fs.s3a.path.style.access", True) \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("spark.hadoop.fs.s3a.ssl.enabled", False) \
#     .getOrCreate()

# # Defina o caminho para os dados raw e trusted
# raw_path = "file:////home/user/to_process/006790cd-5c1b-47b8-9666-045f9dad6846.json"
# trusted_path = "file:////home/user/trusted/006790cd-5c1b-47b8-9666-045f9dad6846.json"

# # Leia os dados raw
# raw_df = spark.read.json(raw_path)

# # Leia os dados trusted
# trusted_df = spark.read.format("delta").load(trusted_path)

# # Realize a operação de MERGE
# from delta.tables import DeltaTable

# deltaTable = DeltaTable.forPath(spark, trusted_path)

# deltaTable.alias("trusted").merge(
#     source = raw_df.alias("raw"),
#     condition = "trusted.id = raw.id"
# ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()


In [ ]:
# from pyspark.sql import SparkSession

# # Crie uma sessão Spark com as configurações do MinIO
# spark = SparkSession.builder \
#     .appName("Read from MinIO") \
#     .config("spark.hadoop.fs.s3a.endpoint", "minio") \
#     .config("spark.hadoop.fs.s3a.access.key", "wFw2nWmmNHNivuqh") \
#     .config("spark.hadoop.fs.s3a.secret.key", "Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs") \
#     .config("spark.hadoop.fs.s3a.path.style.access", True) \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.hadoop.fs.s3a.ssl.enabled", False) \
#     .getOrCreate()

# # Leia um arquivo do MinIO
# df = spark.read.text("s3a://raw/user/006790cd-5c1b-47b8-9666-045f9dad6846.json")

# # Mostre o conteúdo do DataFrame
# df.show()